Creando un Jupyter solo para las migrciones a SQL

In [1]:
import pandas as pd

In [ ]:
pip install openpyxl


In [5]:
# Cargar el archivo Excel
df = pd.read_excel('Book_excel.xlsx')  # Cambia 'tu_archivo.xlsx' por el nombre de tu archivo

# Guardar como CSV
df.to_csv('Book_01.csv', index=False, encoding='utf-8')

In [8]:
# Lo primero es abrir los CSVs de cada API.

# Empezamos con la extracción de Spoti, que es la que contiene todos los datos, ya que para las otras APIs
# se han usado los artistas que arrojaba Spoti.

# Abrir dataframe de spoty para luego hacer la importacion. 
dataframe_spoty_csv = pd.read_csv('../data/dataframes_combinados.csv')

In [ ]:
dataframe_spoty_csv

In [9]:
''' añadimos una columna que vaya autoincrementándose que será el id de la canción y la ponemos
al principio del df'''

dataframe_spoty_csv.insert(0, 'canción_id', range(1, len(dataframe_spoty_csv) + 1))


In [ ]:
dataframe_spoty_csv

In [ ]:
# asignar un ID autoincremental a cada artista único y luego 
# reutilizar ese ID para las canciones del mismo artista.

# Crear el diccionario para almacenar el ID de cada artista
artista_id_map = {}
current_id = 1

# Función para asignar ID al artista
def asignar_id_artista(artista):
    global current_id
    if artista not in artista_id_map:
        artista_id_map[artista] = current_id
        current_id += 1
    return artista_id_map[artista]

# Aplicar la función a cada fila y crear la columna 'artista_id'
dataframe_spoty_csv['artista_id'] = dataframe_spoty_csv['artista'].apply(asignar_id_artista)

print(dataframe_spoty_csv)

In [ ]:
# Comprobando que está bien viendo todas las filas del df.
pd.set_option('display.max_rows', None)
# Esto vuelve a activar la opción de que no se vean todas las columnas.
pd.reset_option('display.max_rows')
dataframe_spoty_csv

# HACER ESTOS PASOS ANTES DE EMPEZAR ✨

In [ ]:
#pip install SQLAlchemy

In [4]:
# Vamos a probar a importar el df a SQL.
# Hemos creado la tabla en SQL con las columnas canción_id, nombre_cancion_album, artista_id,
# tipo_album_cancion, año_lanzamiento, género.

from sqlalchemy import create_engine

In [5]:
user='root'
password='AlumnaAdalab'
host='127.0.0.1'
database= 'MusicStream_PR2_G2'
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(connection_string)

In [13]:
df_spoty_sin_artista = dataframe_spoty_csv[['canción_id', 'nombre_cancion_album', 'artista_id', 'tipo_album_cancion', 'año_lanzamiento', 'género']]

In [ ]:
df_spoty_sin_artista

In [14]:
df_spoty_sin_artista.to_sql(name='Spotify', con=engine, if_exists='append', index=False)

C:\Users\Macarena\AppData\Local\Temp\ipykernel_8220\3052401549.py:1: UserWarning: The provided table name 'Spotify' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_spoty_sin_artista.to_sql(name='Spotify', con=engine, if_exists='append', index=False)


-1

In [36]:
# Vamos a hacer lo mismo para crear una tabla que contenga artistas con sus IDs, que en ppio. 
# contendrá artistas duplicados ya que viene la tabla anterior con las canciones y sus artistas.

# creamos un df nuevo con las columnas que queremos y ponemos el drop_duplicates para que elimine los duplicados.

df_artistas_spoty = df_spoty_sin_artista = dataframe_spoty_csv[['artista', 'artista_id']].drop_duplicates()

In [37]:
df_artistas_spoty.to_sql(name='Artistas', con=engine, if_exists='append', index=False)
# No deja porque artista_id es PK. Habrá que hacer una copia de la tabla de Spoti

899

In [ ]:
#añadir dataframe de MB1 para ver si soluciona los artistas con distinto nombre
dataframe_MB1_csv = pd.read_csv('../data/dataframe_MB1.csv')
dataframe_MB1_csv.to_sql(name='musicbrainz1', con=engine, if_exists='append', index=False)

-1

In [ ]:
dataframe_mb_csv = pd.read_csv('../data/dataframe_MB.csv')
dataframe_mb_csv.tosql(name='informacion_artistas', con=engine, if_exists='append', index=False)

#### Como modificar 'None' a Null en la base de datos

In [ ]:
import mysql.connector
from time import sleep

# Configuración de la conexión
conexion = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database="MusicStream_PR2_G2"
)
cursor = conexion.cursor()

# Obtener todas las tablas
cursor.execute("SHOW TABLES")
tablas = cursor.fetchall()

for (tabla,) in tablas:
    cursor.execute(f"SHOW COLUMNS FROM {tabla}")
    columnas = cursor.fetchall()

    for (columna, tipo, _, _, _, _) in columnas:
        if tipo.startswith(('char', 'varchar', 'text')):
            try:
                sql = f"UPDATE {tabla} SET {columna} = NULL WHERE {columna} = 'None'"
                cursor.execute(sql)
                print(f"Actualizados valores 'None' en {tabla}.{columna}")
            except mysql.connector.Error as err:
                print(f"Error al actualizar en {tabla}.{columna}: {err}")
                # Esperar y reintentar
                sleep(2)  # Espera 2 segundos antes de reintentar
                continue

# Confirmar cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()
